In [2]:
import requests
import json
from pandas import DataFrame, to_datetime

import sys
sys.path.append("../../")

import datetime as dt
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
import seaborn as sb

# from helper import set_datetime_index, exp_time_data

In [3]:
# API URL
API_URL = "https://openapi.naver.com/v1/datalab/search"

In [8]:
client_id = "rETvIsXmOenuP6unjdAz"
client_secret = "z7J2r3pSHd"

In [9]:
session = requests.Session()
session.headers.update({
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
})

In [10]:
data = {
    "startDate": "2020-01-01",   # 2016년 1월 1일부터 조회 시작 가능
    "endDate": "2023-07-31",
    "timeUnit": "date",         # date: 일간, week: 주간, month: 월간
    "keywordGroups": [
        {
            "groupName": "코로나",  # 검색주제
            "keywords": ["코로나", "Covid19", "격리", "확진자"]   # 주제와 관련된 검색어
        }
    ]
}

In [11]:
r = session.post(API_URL, data=json.dumps(data))

if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)

r.encoding = "utf-8"
mydict = json.loads(r.text)
mydict

{'startDate': '2020-01-01',
 'endDate': '2023-07-31',
 'timeUnit': 'date',
 'results': [{'title': '코로나',
   'keywords': ['코로나', 'Covid19', '격리', '확진자'],
   'data': [{'period': '2020-01-01', 'ratio': 0.00869},
    {'period': '2020-01-02', 'ratio': 0.01115},
    {'period': '2020-01-03', 'ratio': 0.01228},
    {'period': '2020-01-04', 'ratio': 0.01182},
    {'period': '2020-01-05', 'ratio': 0.0096},
    {'period': '2020-01-06', 'ratio': 0.01228},
    {'period': '2020-01-07', 'ratio': 0.01192},
    {'period': '2020-01-08', 'ratio': 0.0118},
    {'period': '2020-01-09', 'ratio': 0.05425},
    {'period': '2020-01-10', 'ratio': 0.04167},
    {'period': '2020-01-11', 'ratio': 0.01865},
    {'period': '2020-01-12', 'ratio': 0.01171},
    {'period': '2020-01-13', 'ratio': 0.01305},
    {'period': '2020-01-14', 'ratio': 0.01334},
    {'period': '2020-01-15', 'ratio': 0.01238},
    {'period': '2020-01-16', 'ratio': 0.01632},
    {'period': '2020-01-17', 'ratio': 0.01752},
    {'period': '2020-01-1

In [12]:
df = DataFrame(mydict['results'][0]['data'])
df

,period,ratio
0,2020-01-01,0.00869
1,2020-01-02,0.01115
2,2020-01-03,0.01228
3,2020-01-04,0.01182
4,2020-01-05,0.00960
...,...,...
1303,2023-07-27,1.11399
1304,2023-07-28,0.91158
1305,2023-07-29,0.65329
1306,2023-07-30,0.79660


In [13]:
df.dtypes

period     object
ratio     float64
dtype: object

In [15]:
df['period'] = to_datetime(df['period'])
# df2 = set_datetime_index(df, 'period')
# df2